**Uladzislau Darhevich**

**ud108519@student.sgh.waw.pl**

# Zadanie 1
Utwórz zbiór danych, zawierający ID (`customerID`) klienta oraz miesięczny abonament (`MonthlyCharges`)

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
import numpy as np

In [0]:
d = spark.read.format('com.databricks.spark.csv').options(inferSchema="true",header='true', delimiter=';').load('/FileStore/tables/telco3.csv')
d.select("customerID", "MonthlyCharges").show()

+----------+--------------+
customerID|MonthlyCharges|
+----------+--------------+
7590-VHVEG| 29.85|
5575-GNVDE| 56.95|
3668-QPYBK| 53.85|
7795-CFOCW| 42.3|
9237-HQITU| 70.7|
9305-CDSKC| 99.65|
1452-KIOVK| 89.1|
6713-OKOMC| 29.75|
7892-POOKP| 104.8|
6388-TABGU| 56.15|
9763-GRSKD| 49.95|
7469-LKBCI| 18.95|
8091-TTVAX| 100.35|
0280-XJGEX| 103.7|
5129-JLPIS| 105.5|
3655-SNQYZ| 113.25|
8191-XWSZG| 20.65|
9959-WOFKT| 106.7|
4190-MFLUW| 55.2|
4183-MYFRB| 90.05|
+----------+--------------+
only showing top 20 rows

Wyznacz liczności klientów względem rodzaju podpisanej umowy (`Contract`). Zaprezentuj na wykresie.

In [0]:
display(d.groupBy("Contract").count())

Contract,count
Month-to-month,3875
One year,1473
Two year,1695


Ilu klientów posiada wiele linii telefonicznych (`MultipleLines`)?

In [0]:
print("{} klientów posiada wiele linii telefonicznych.". format(d.groupBy("MultipleLines")\
                                                               .count().filter(f.col("MultipleLines") == "Yes")\
                                                               .collect()[0]["count"]))


2971 klientów posiada wiele linii telefonicznych.

#Zadanie 2 
Jaka jest minimalna i maksymalna długość stażu (`tenure`)?

In [0]:
d.agg(f.min("tenure").alias("Minimalna długość stażu"),
     f.max("tenure").alias("Maksymalna długość stażu")).show()

+-----------------------+------------------------+
Minimalna długość stażu|Maksymalna długość stażu|
+-----------------------+------------------------+
 0| 72|
+-----------------------+------------------------+

Ile miesięcznie średnio płacą mężczyźni a ile kobiety (`gender`)?

In [0]:
d.groupBy('gender').agg(f.bround(f.avg('MonthlyCharges'), 2)\
                        .alias("Średnie miesięczne opłaty, $"))\
                        .show()

+------+----------------------------+
gender|Średnie miesięczne opłaty, $|
+------+----------------------------+
Female| 65.2|
 Male| 64.33|
+------+----------------------------+

Czy klienci, którzy podpisali umowę na rok (`Contract`), płacą więcej od pozostałych (średnio)? ***Według tabeli - nie***

In [0]:
d.groupBy('Contract').agg(f.bround(f.avg('MonthlyCharges'), 2)\
                        .alias("Średni rozmiar opłaty, $"))\
                        .show()

+--------------+------------------------+
 Contract|Średni rozmiar opłaty, $|
+--------------+------------------------+
Month-to-month| 66.4|
 One year| 65.05|
 Two year| 60.77|
+--------------+------------------------+

Utwórz zmienną, będącą ilorazem `TotalCharges` i `tenure`

In [0]:
d = d.withColumn("TotalCharges/tenure", d.TotalCharges / d.tenure)

# Zadanie 3 
Utwórz zmienną churn2, która przyjmuje tylko wartości 0 (dla Churn= No) i 1 (dla Churn= Yes)

In [0]:
d = d.withColumn('churn2', f.when(d.Churn == "No", 0).otherwise(1))

Wykonaj kroki:
- a)Podziel klientów na przedziały związane z lojalnością (tenure). 
  - Klientów o stażu krótszym niż 6 miesięcy zignoruj:
  - 6-12 m-cy
  - 12-36 m-cy
  - 36 - ... m-cy
- b)Zapisz wyniki w bazie danych Hive, w tabeli tenureResults
- c)Przeanalizuj zjawisko Churn w wyznaczonych grupach

In [0]:
d = d.withColumn(
    'segment',
    f.when((d.tenure >= 6) & (d.tenure < 12), "low").\
    when((d.tenure >= 12) & (d.tenure < 36), "medium").\
    when(d.tenure >= 36, "high"))

In [0]:
d.write.mode("overwrite").saveAsTable("tenureResults")

In [0]:
%sql
select segment, COUNT(churn) as Num, ROUND(AVG(MonthlyCharges), 2) as Average_bill
from tenureResults
where segment IS NOT NULL
group by segment

segment,Num,Average_bill
low,698,58.94
high,3051,71.9
medium,1923,62.8


# Zadanie 4
Wyznacz 5 klientów, o najwyższych płatnościach całkowitych (`TotalCharges`)

In [0]:
%sql
select customerID, TotalCharges
from tenureResults
order by TotalCharges desc
limit 5

customerID,TotalCharges
9093-FPDLG,999.9
4536-PLEQY,999.8
5899-MQZZL,999.45
6051-PTVNS,998.1
8249-THVEC,997.75


Wyznacz średnie opłaty abonamentowe (`monthlyCharges`) dla klientów którzy odeszli i dla klientów, którzy nie odeszli (`Churn`)

In [0]:
%sql
select churn, round(AVG(monthlyCharges), 2) as average_bill
from tenureResults
GROUP BY churn

churn,average_bill
No,61.27
Yes,74.44


Wyznacz średnie oraz maksymalne wartości opłat dla kontraktów rocznych i miesięcznych (`monthlyCharges`)

In [0]:
%sql
select Contract, round(avg(monthlyCharges), 2) as average_bill, round(max(monthlyCharges), 2) as max_bill
from tenureResults
group by Contract

Contract,average_bill,max_bill
Month-to-month,66.4,117.45
One year,65.05,118.6
Two year,60.77,118.75


Utwórz kolumnę genderShort, która przyjmuje wartości M i F

In [0]:
%sql
SELECT customerID, gender, 
CASE 
    WHEN gender = 'Male' THEN 'M' ELSE 'F' END as genderShort
FROM tenureResults;

customerID,gender,genderShort
7590-VHVEG,Female,F
5575-GNVDE,Male,M
3668-QPYBK,Male,M
7795-CFOCW,Male,M
9237-HQITU,Female,F
9305-CDSKC,Female,F
1452-KIOVK,Male,M
6713-OKOMC,Female,F
7892-POOKP,Female,F
6388-TABGU,Male,M
